In [ ]:
import os
import pandas as pd
print(os.getcwd())
#from google.colab import drive
#MOUNTPOINT = '/content/gdrive'
#drive.mount(MOUNTPOINT)
#os.chdir('/content/gdrive/MyDrive/cs522')
os.chdir('/content/drive/MyDrive/cs522') # drive is permanently mounted now hurray
print(os.getcwd())
!pip install minisom
from minisom import MiniSom

/content
/content/drive/MyDrive/cs522
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for minisom: filename=MiniSom-2.3.0-py3-none-any.whl size=9016 sha256=a8102b72bb276a4a86e3e1c04c1f2395115f284dcdf36980fe228ff6e18cd6d5
  Stored in directory: /root/.cache/pip/wheels/6d/4e/9e/a95c14a232a196c22d9c04b221ff5d25461a1a4c55339c61db
Successfully built minisom


Kaggle stopped working so here we are
Goal is to create the data to be used in clustering the precipitation data for each survey year + route number combo.


First we have to associate each route with a file based on closest gps coordinates; and then we need a function that can return a series of data based on coordinates and time period; we'll transform the data in another fn

All of the ordinal dates in consolidated.csv are already transformed, so we have to use Runs.csv.

In [ ]:
import numpy
from sklearn.preprocessing import MinMaxScaler

def distsqrd(x1,y1,x2,y2) : 
  return ((x2-x1)*(x2-x1)) + ((y2-y1)*(y2-y1)) # return distance squared

def returnClosestFile(lat, lng, arraywithfilename) :
  winner = 99999999
  winningIndex = 0
  currentIndex = 0
  for x in arraywithfilename :
    current = distsqrd(lng,lat,x[1], x[0])
    if current < winner :
      winner = current
      winningIndex = currentIndex
    currentIndex += 1
  return arraywithfilename[winningIndex][2] # return file name

def returnTimeSeriesBasedOnTimeRange(dataframeFromPrecipFile, startDate, endDate) :
  tempDataframe = dataframeFromPrecipFile.set_index('DATE')
  tempNumpy = tempDataframe[startDate : endDate].PRCP.to_numpy()
  tempNumpy = numpy.nan_to_num(tempNumpy)
  return tempNumpy

# read a precipitation file and return DataFrame
def readprecip(filename) :
  return pd.read_csv(filename, parse_dates=['DATE'])

#testframe = returnTimeSeriesBasedOnTimeRange(readprecip('3163000.csv'), '2001-01-01', '2001-01-15')
#print(testframe)

# returns a dictionary that matches run id to survey date
def readRunsRetrieveDates(filepath) :
  runsDF = pd.read_csv(filepath, parse_dates=['SurveyDate']) # I'm glad for the magic that is the date parser
  #print(runsDF.head())
  #print(runsDF.dtypes)
  returnValue = {}
  for index, row in runsDF.iterrows() :
    returnValue[row.RunID] = row.SurveyDate
  return returnValue

#runsdict = readRunsRetrieveDates('Runs.csv')
#print(runsdict[189])

# returns a DataFrame which can use .loc[route number] to retrieve coordinates
def readRouteUnitCoordinates(filepath) :
  tmp = pd.read_csv(filepath)
  return tmp.set_index('RouteNumber')

#uc = readRouteUnitCoordinates('RouteUnitCoordinates.csv')
#print(uc.loc[250122])

def readConsolidated(filepath) :
  return pd.read_csv(filepath)

coordarray = [[ 44.53845502, -93.83408473, '3163000.csv'],
       [ 39.0125231 , -79.95821355, '3163009.csv'],
       [ 30.96771759, -84.4666357, '3163013.csv' ],
       [ 43.29577165, -71.80487573, '3163016.csv'],
       [ 31.72070082, -90.2132503, '3155781-mod.csv' ],
       [ 39.22193018, -76.35985149, '3155785.csv'],
       [ 39.76749934, -86.2771057, '3155808.csv' ],
       [ 35.87956162, -76.99912934, '3155861.csv'],
       [ 38.11810374, -92.30672146, '3155871.csv'],
       [ 34.73908715, -83.79985733, '3155873.csv'],
       [ 45.23027738, -68.96514196, '3155888.csv'],
       [ 47.02496426, -95.2990422, '3155894.csv' ],
       [ 40.91172204, -74.4773553, '3155906.csv' ],
       [ 30.60071905, -94.11526452, '3155996.csv'],
       [ 28.08420413, -81.70594462, '3156005.csv'],
       [ 33.11577242, -81.17899961, '3156014.csv'],
       [ 41.49745322, -93.02686429, '3156020.csv'],
       [ 47.33513513, -92.26729451, '3156028.csv'],
       [ 35.46922886, -88.55395709,'3156036.csv'],
       [ 35.63904872, -80.02362788, '3156042.csv']]
coorddict = {}

# returns a dictionary mapping file name to DataFrame
def readAllPrecipFiles(coray) :
  filedict = {}
  for x in coray :
    # associate file name with the data from that file
    filedict[x[2]] = readprecip(x[2])
  return filedict

def normalizeIV (interpVals) :
  llv2 = []
  for x in interpVals : # each element has to be turned into a list
    llv2.append([x])
    t2 = MinMaxScaler().fit_transform(llv2)
    t2 = t2.reshape(len(t2))
  #print(t2) # looks fine
  return t2

def outputfn(coray) :
  successes = 0
  attempts = 0
  # return value, dictionary route then year then precip data
  routeThenYearOut = {}
  # read all precipitation files
  precips = readAllPrecipFiles(coray)
  # read RouteUnitCoordinates.csv
  routeToCoordDF = readRouteUnitCoordinates('RouteUnitCoordinates.csv')
  # read Runs.csv
  runIDToDateDict = readRunsRetrieveDates('Runs.csv')
  # read consolidated.csv
  concsv = readConsolidated('consolidated.csv')
  #print(concsv.describe().T)
  #print(concsv.head())
  #print(concsv.tail(10))
  # iterate through it
  #for index, row in concsv.iterrows() :
  for index in range(6118) :
    row = concsv.iloc[index]
    #print(index)
    attempts += 1
    if pd.isna(row.RunID1) :
      continue
    if pd.isna(row.RunID2) :
      continue
    if pd.isna(row.RunID3) :
      continue
    # add dict if needed
    if row.RouteNumber not in routeThenYearOut :
      routeThenYearOut[row.RouteNumber] = {}
    # determine the first and last runs of a given year+route
    firstrun = runIDToDateDict[row.RunID1]
    lastrun = runIDToDateDict[row.RunID3]
    if not pd.isna(row.RunID4) :
      lastrun = runIDToDateDict[row.RunID4]
    # find the coordinates of the route
    if row.RouteNumber in routeToCoordDF.index :
      coordSeries = routeToCoordDF.loc[row.RouteNumber]
    else :
      continue
    # find the closest weather file, retrieve the previously read-in precip data
    precipFileName = returnClosestFile(coordSeries.lat, coordSeries.lon, coray)
    precipDF = precips[precipFileName]
    # retrieve data between first and last runs - future work would be to retrieve data before the first run as well
    periodSeries = returnTimeSeriesBasedOnTimeRange(precipDF, firstrun, lastrun)
    if len(periodSeries) == 0 :
      #print(precipDF,firstrun,lastrun)
      continue
    #print(precipFileName, coordSeries.lat, coordSeries.lon, periodSeries) # run a litte sanity check on the result
    # apparently numpy was talking about the x coordinates have to be monotonically increasing, not the values, so I can use it
    x = []
    centi = len(periodSeries) / 100
    for i in range(100) :
      # we have to fit whatever data range size we have into 100 units, chosen because it's a nice round number and is close to the actual data series lengths in some cases
      x.append(i*centi)
    xp = []
    for i in range(len(periodSeries)) :
      xp.append(i)
    #if True in numpy.isnan(periodSeries) :
    #  print(periodSeries, precipFileName, firstrun, lastrun)  # tracking down missing values
    #  break
    interpolatedValues = numpy.interp(x,xp,periodSeries)
    routeThenYearOut[row.RouteNumber][row.SurveyYear] = normalizeIV(interpolatedValues)
    successes += 1
    #print(interpolatedValues)
    #break
  print(attempts,successes)
  return routeThenYearOut

try :
  routeThenYearPrecipatation = outputfn(coordarray)
except Exception as e:
  print(e)
print(len(routeThenYearPrecipatation))

6118 4887
1120


In [ ]:
# now that we have the time series, we can run MiniSOM on them
normalizedScoresOverTime = []
routeList = []
yearList = []
for routeNumber, yearDict in routeThenYearPrecipatation.items() :
  for yearKey, scoreList in yearDict.items() :
    normalizedScoresOverTime.append(scoreList)
    routeList.append(routeNumber)
    yearList.append(yearKey)
# Trying to get a map such that not every data point is its own centroid
map_side_length = 30
# default arguments, starting out matching side length to ~square root of number of items, we'll go from there
asom = MiniSom(map_side_length, map_side_length, len(normalizedScoresOverTime[0])) 
asom.random_weights_init(normalizedScoresOverTime)
asom.train(normalizedScoresOverTime, 50000)
# 
print("training done")


training done


In [ ]:
# With training complete, we need to store the winning centroids so we can do associative analysis later
#for i in range(len(normalizedScoresOverTime)) :
  #print(asom.winner(normalizedScoresOverTime[i]))

# I'm still seeing a lot of 1s and 2s, so I might need to decrease the map size again
for (row,column), value  in asom.win_map(normalizedScoresOverTime).items() :
  print(row,column,len(value))

In [ ]:
# need to write all this to a file so we can run association analysis
xlist = []
ylist = []
for x in normalizedScoresOverTime :
  (xwin, ywin) = asom.winner(x)
  xlist.append(xwin)
  ylist.append(ywin)
  #print(asom.winner(x)) # printed out looks okay
  #print(xwin,ywin)
  #break
outDataFrame = pd.DataFrame({
    'RouteNumber': routeList,
    'SurveyYear': yearList,
    'x': xlist,
    'y': ylist})
outDataFrame.to_csv('precipWinners.csv')